In [1]:
import model
import torch
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import os

In [2]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
network = model.get_network(model.Network.GROUP_NORM).to(device)
summary(network, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
              ReLU-2           [-1, 10, 28, 28]               0
         GroupNorm-3           [-1, 10, 28, 28]              20
         Dropout2d-4           [-1, 10, 28, 28]               0
            Conv2d-5           [-1, 16, 28, 28]           1,456
              ReLU-6           [-1, 16, 28, 28]               0
         GroupNorm-7           [-1, 16, 28, 28]              32
         Dropout2d-8           [-1, 16, 28, 28]               0
         MaxPool2d-9           [-1, 16, 14, 14]               0
           Conv2d-10            [-1, 8, 14, 14]             136
             ReLU-11            [-1, 8, 14, 14]               0
        GroupNorm-12            [-1, 8, 14, 14]              16
    

/content/model.py:114: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [3]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        #transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [4]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0

    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        y_pred = model(data)
        pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        pbar.set_description(desc= f'Epoch={epoch} Loss={loss.item()} batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}%')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [5]:
network = model.get_network(model.Network.GROUP_NORM).to(device)
optimizer = optim.SGD(network.parameters(), lr=0.01, momentum=0.9)
#scheduler = StepLR(optimizer, step_size=5, gamma=0.1)


for epoch in range(1, 16):
    train(network, device, train_loader, optimizer, epoch)
    test(network, device, test_loader)

Epoch=1 Loss=0.12392862141132355 batch_id=937 Accuracy=88.02%: 100%|██████████| 938/938 [00:25<00:00, 36.14it/s]



Test set: Average loss: 0.0681, Accuracy: 9808/10000 (98.08%)



Epoch=2 Loss=0.04138115793466568 batch_id=937 Accuracy=97.20%: 100%|██████████| 938/938 [00:22<00:00, 41.00it/s]



Test set: Average loss: 0.0516, Accuracy: 9847/10000 (98.47%)



Epoch=3 Loss=0.08628048002719879 batch_id=937 Accuracy=97.67%: 100%|██████████| 938/938 [00:23<00:00, 40.74it/s]


...


Test set: Average loss: 0.0302, Accuracy: 9907/10000 (99.07%)



Epoch=14 Loss=0.09740747511386871 batch_id=937 Accuracy=98.87%: 100%|██████████| 938/938 [00:23<00:00, 39.86it/s]



Test set: Average loss: 0.0289, Accuracy: 9912/10000 (99.12%)



Epoch=15 Loss=0.014592999592423439 batch_id=937 Accuracy=98.89%: 100%|██████████| 938/938 [00:23<00:00, 39.70it/s]



Test set: Average loss: 0.0269, Accuracy: 9920/10000 (99.20%)



In [6]:
network = model.get_network(model.Network.LAYER_NORM).to(device)
summary(network, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
              ReLU-2           [-1, 10, 28, 28]               0
         LayerNorm-3           [-1, 10, 28, 28]          15,680
         Dropout2d-4           [-1, 10, 28, 28]               0
            Conv2d-5           [-1, 16, 28, 28]           1,456
              ReLU-6           [-1, 16, 28, 28]               0
         LayerNorm-7           [-1, 16, 28, 28]          25,088
         Dropout2d-8           [-1, 16, 28, 28]               0
         MaxPool2d-9           [-1, 16, 14, 14]               0
           Conv2d-10            [-1, 8, 14, 14]             136
             ReLU-11            [-1, 8, 14, 14]               0
        LayerNorm-12            [-1, 8, 14, 14]           3,136
        Dropout2d-13            [-1, 8, 14, 14]               0
           Conv2d-14           [-1, 16,

/content/model.py:211: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [7]:
network = model.get_network(model.Network.LAYER_NORM).to(device)
optimizer = optim.SGD(network.parameters(), lr=0.01, momentum=0.9)
#scheduler = StepLR(optimizer, step_size=5, gamma=0.1)


for epoch in range(1, 16):
    train(network, device, train_loader, optimizer, epoch)
    test(network, device, test_loader)

Epoch=1 Loss=0.17002348601818085 batch_id=937 Accuracy=90.24%: 100%|██████████| 938/938 [00:22<00:00, 41.14it/s]



Test set: Average loss: 0.0764, Accuracy: 9766/10000 (97.66%)



Epoch=2 Loss=0.05211994796991348 batch_id=937 Accuracy=97.21%: 100%|██████████| 938/938 [00:23<00:00, 40.28it/s]



Test set: Average loss: 0.0530, Accuracy: 9831/10000 (98.31%)



Epoch=3 Loss=0.10989350825548172 batch_id=937 Accuracy=97.81%: 100%|██████████| 938/938 [00:23<00:00, 40.64it/s]


...


Test set: Average loss: 0.0280, Accuracy: 9914/10000 (99.14%)



Epoch=14 Loss=0.01370526384562254 batch_id=937 Accuracy=98.98%: 100%|██████████| 938/938 [00:23<00:00, 40.18it/s]



Test set: Average loss: 0.0287, Accuracy: 9904/10000 (99.04%)



Epoch=15 Loss=0.015676043927669525 batch_id=937 Accuracy=98.98%: 100%|██████████| 938/938 [00:22<00:00, 40.91it/s]



Test set: Average loss: 0.0269, Accuracy: 9911/10000 (99.11%)

